1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [99]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from gensim.models import Word2Vec
from keras.models import load_model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from tensorflow.keras.callbacks import EarlyStopping

In [100]:
tashkeel = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
arabicCharacters = "ىعظحرسيشضقثلصطكآماإهزءأفؤغجئدةخوبذتن"
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = [".", "،", ":", "؛", "-", "؟"]
validCharacters= tashkeel + list(arabicCharacters) + punctuations
charcters_without_tashkeel = list(arabicCharacters) + punctuations
print(validCharacters)
print(charcters_without_tashkeel)

['ْ', 'ّ', 'ٌ', 'ٍ', 'ِ', 'ً', 'َ', 'ُ', 'ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن', '.', '،', ':', '؛', '-', '؟']
['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن', '.', '،', ':', '؛', '-', '؟']


In [101]:
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))

<class 'str'>


In [102]:
print(len(f))
# arabic_stopwords = set(nltk.corpus.stopwords.words("arabic"))
#regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
processedData = re.findall(characters_regex,f)
processedData = " ".join(processedData)
processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
# processedData = ' '.join(word for word in processedData.split() if word not in arabic_stopwords) # remove stopwors from text
print(len(processedData))
print(processedData[0])

18515271
17997403
ق


In [103]:
w = open('processed.txt', 'w', encoding='utf-8')

w.write(processedData)
w.close()

In [104]:
#remove diacritics
without_diacritics= araby.strip_diacritics(processedData)
print(len(without_diacritics))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')
w2.write(without_diacritics)
w2.close()

10711561


In [105]:
#take the procrssed text and separate it into sentences
fileRead=open('processed.txt', encoding='utf-8')
lines=fileRead.readlines()
longString1=' '.join(lines)
longStringSplited=re.sub(r"\n", ' ', longString1)
without_diacritics_splitted=re.sub(r"\n", ' ', without_diacritics)
without_diacritics_splitted=without_diacritics_splitted.split('.')
#the dot itself got removed in the process
longStringSplited=longStringSplited.split('.')
print(len(longStringSplited))
w2 = open('withoutDiacritics.txt', 'w', encoding='utf-8')

w2.write(without_diacritics)

w3 = open('processed_sentences_separated.txt', 'w', encoding='utf-8')
list_of_sentences=[]
for line in longStringSplited:
 w3.write(line)

for line in without_diacritics_splitted: #list of lists of sentences splitted in words without diacratics
 list_of_sentences.append(line.split(" "))

 #now longStringSplited is the list of procrssed text without practices and numbers and dots 
 #the rest of the punctuation still there  
# print(list_of_sentences[0:50]) 
 


40836


In [106]:
# now we want to create a list of words and a crossbonding list of punctuation for each word 

#first remove all puncturation 

fileRead=open('processed_sentences_separated.txt', encoding='utf-8')
lines=fileRead.readlines()
print(type(lines))
longString=' '.join(lines)             
longString=re.sub(r"\n", ' ', longString)
if longString == longString1:
     print("true")
punctuations = ["،", ":", "؛", "-", "؟"]
for element in punctuations:
     longString=re.sub(element, '', longString)



<class 'list'>


In [107]:
# now the varable called longString has a single string with all the processed words in it 
listOfwordsWith_Diacritics=list()
listOfwordsWith_NoDiacritics=list()

without_diacritics= araby.strip_diacritics(longString)


without_diacritics=re.sub(r"\s+", ' ', without_diacritics)

listOfwordsWith_NoDiacritics=without_diacritics.split(" ")


listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")





In [108]:
#now that we have two separated lists we need to get the diacritics list 


listofDiacritrcs_ToWord=list()
temp=list()
counter=0
flag=0
for word in listOfwordsWith_Diacritics:
    while counter<len(word):
     if word[counter] in arabicDictionary: #checking if the character is a letter
      if (counter+1)<len(word):
        #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
        if word[counter +1] in arabicDictionary: 
          temp.append("")
          counter+=1
          continue
      counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
      continue
     else:
      if (counter+1)<len(word):
        if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
          temp.append(word[counter]+word[counter+1])
          counter+=2
          continue
      temp.append(word[counter]) #if the current character only is the diacritics add it to the list
      counter+=1    
    listofDiacritrcs_ToWord.append(temp.copy())     
    temp.clear() 
    counter=0
          
          
    
        


print(listOfwordsWith_Diacritics[0:100])
print(listOfwordsWith_NoDiacritics[0:100])

['قَوْلُهُ', 'أَوْ', 'قَطَعَ', 'الْأَوَّلُ', 'يَدَهُ', 'إلَخْ', 'قَالَ', 'الزَّرْكَشِيُّ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُهُ', 'بِلَفْظٍ', 'يَقْتَضِيه', 'كَإِنْكَارِ', 'غَيْرِ', 'حَدِيثٍ', 'بِالْإِسْلَامِ', 'وُجُوبَ', 'مَا', 'عُلِمَ', 'وُجُوبُهُ', 'مِنْ', 'الدِّينِ', 'ضَرُورَةً', 'كَإِلْقَاءِ', 'مُصْحَفٍ', 'بِقَذَرٍ', 'وَشَدِّ', 'زُنَّارٍ', 'ابْنُ', 'عَرَفَةَ', 'قَوْلُ', 'ابْنِ', 'شَاسٍ', 'أَوْ', 'بِفِعْلٍ', 'يَتَضَمَّنُهُ', 'هُوَ', 'كَلُبْسِ', 'الزُّنَّارِ', 'وَإِلْقَاءِ', 'الْمُصْحَفِ', 'فِي', 'صَرِيحِ', 'النَّجَاسَةِ', 'وَالسُّجُودِ', 'لِلصَّنَمِ', 'وَنَحْوِ', 'ذَلِكَ', 'وَسِحْرٍ', 'مُحَمَّدٌ', 'قَوْلُ', 'مَالِكٍ', 'وَأَصْحَابِهِ', 'أَنَّ', 'السَّاحِرَ', 'كَافِرٌ', 'بِاَللَّهِ', 'تَعَالَى', 'قَالَ', 'مَالِكٌ', 'هُوَ', 'كَالزِّنْدِيقِ', 'إذَا', 'عَمِلَ', 'السِّحْرَ', 'بِنَفْسِهِ', 'قُتِلَ', 'وَلَمْ', 'يُسْتَتَبْ', 'قَوْلُهُ', 'لِعَدَمِ', 'مَا', 'تَتَعَلَّقُ', 'إلَخْ', 'أَيْ', 'الْوَصِيَّةُ', 'قَوْلُهُ', 'مَا', 'مَرَّ', 'أَيْ', 'قُبَيْلَ', 'قَوْلِ', 'الْمَتْنِ', 'لَغَتْ', 'وَلَوْ', 'اقْت

In [109]:
print(len(listOfwordsWith_NoDiacritics))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2102046
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

2-Word Embeddings

In [110]:
# Train Word2Vec model
Word2Vec_model = Word2Vec(list_of_sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1)
Word2Vec_model.train(list_of_sentences,total_examples=len(listOfwordsWith_NoDiacritics),epochs=10)

(18728263, 22684760)

In [111]:
word_embeddings = {word: Word2Vec_model.wv[word] for word in Word2Vec_model.wv.index_to_key}

3-Building the LSTM model for character level classifier

In [112]:
for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
# print(listofDiacritrcs_ToWord[0:100])
Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object)
print(str(X_train.shape))

(1, 2102046)


In [113]:
# Tokenize the input words and diacritics
word_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
word_tokenizer.fit_on_texts(listOfwordsWith_NoDiacritics)
word_sequences = word_tokenizer.texts_to_sequences(listOfwordsWith_NoDiacritics)

diacritic_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
diacritic_tokenizer.fit_on_texts([''.join(d) for d in listofDiacritrcs_ToWord])
diacritic_sequences = diacritic_tokenizer.texts_to_sequences([''.join(d) for d in listofDiacritrcs_ToWord])

# Pad sequences to have the same length
max_len = max(max(len(seq) for seq in word_sequences), max(len(seq) for seq in diacritic_sequences))
padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len, padding='post')

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_word_sequences, np.expand_dims(padded_diacritic_sequences, -1), epochs=10, batch_size=32)

# Save the model for later use
model.save("diacritic_prediction_model.h5")

# Now, you can use the trained model to predict diacritics for new Arabic words
def predict_diacritics(model, word):
    word_sequence = word_tokenizer.texts_to_sequences([word])
    padded_word_sequence = pad_sequences(word_sequence, maxlen=max_len, padding='post')
    predicted_diacritic_sequence = model.predict(padded_word_sequence)
    predicted_diacritic_sequence = np.argmax(predicted_diacritic_sequence, axis=-1)
    predicted_diacritic = diacritic_tokenizer.sequences_to_texts(predicted_diacritic_sequence)
    return predicted_diacritic[0]




Epoch 1/10
65689/65689 [==============================] - 1123s 17ms/step - loss: 0.1175 - accuracy: 0.9577
Epoch 2/10
65689/65689 [==============================] - 1141s 17ms/step - loss: 0.0916 - accuracy: 0.9656
Epoch 3/10
65689/65689 [==============================] - 1100s 17ms/step - loss: 0.0883 - accuracy: 0.9666
Epoch 4/10
65689/65689 [==============================] - 1088s 17ms/step - loss: 0.0867 - accuracy: 0.9671
Epoch 5/10
65689/65689 [==============================] - 1099s 17ms/step - loss: 0.0858 - accuracy: 0.9674
Epoch 6/10
65689/65689 [==============================] - 1236s 19ms/step - loss: 0.0852 - accuracy: 0.9676
Epoch 7/10
65689/65689 [==============================] - 1646s 25ms/step - loss: 0.0847 - accuracy: 0.9677
Epoch 8/10
65689/65689 [==============================] - 2616s 40ms/step - loss: 0.0843 - accuracy: 0.9678
Epoch 9/10
65689/65689 [==============================] - 1226s 19ms/step - loss: 0.0841 - accuracy: 0.9679
Epoch 10/10
65689/65689 [===

C:\Users\admin\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [117]:
# Example usage
# input = "الذي علم بالقلم"
input= "ذهب علي إلى الشاطئ"
model = load_model("diacritic_prediction_model.h5")

# input = "قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر"
input_word = input.split(' ')
word_vectors = [Word2Vec_model.wv[word] for word in input_word if word in Word2Vec_model.wv]
# print(word_vectors)
outputString=""
print("number of words",len(input_word))
for word in input_word: 
  predicted_diacritics1 = predict_diacritics(model, word)
  #print(predicted_diacritics1.split(' '))
  predictedDiacritics=(predicted_diacritics1.split('  '))
  temp=list()
  for diacrtic in predictedDiacritics:
    # print(diacrtic)
    # print(len(diacrtic))
    if ' ' in diacrtic:
      # print("after space removel")
      diacrtic=diacrtic.replace(' ','')
      # print(diacrtic)
      # print(len(diacrtic))
      temp.append(diacrtic)
      continue
    temp.append(diacrtic)
  predictedDiacritics=temp.copy()
  tempString=""
  for i in range(len(word)):
    tempString+=word[i]
    if i<len(predictedDiacritics):
      tempString+=predictedDiacritics[i]
  print(tempString)
  tempString+=" "
  outputString+=tempString

print(outputString)  


number of words 4
1/1 [==============================] - 0s 363ms/step
ذَهَبَ
1/1 [==============================] - 0s 22ms/step
عَلَيَّ
1/1 [==============================] - 0s 28ms/step
إلَى
1/1 [==============================] - 0s 21ms/step
الَّْشاِطِئ
ذَهَبَ عَلَيَّ إلَى الَّْشاِطِئ 
